In [1]:
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
from torch.autograd import grad
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import matplotlib.patches as patches

class BeamPINN(nn.Module):
    def __init__(self, layers):
        super(BeamPINN, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(len(layers) - 1):
            self.layers.append(nn.Linear(layers[i], layers[i+1]))

    def forward(self, x, t):
        inputs = torch.cat([x, t], dim=1)
        for i, layer in enumerate(self.layers[:-1]):
            inputs = torch.tanh(layer(inputs))
        output = self.layers[-1](inputs)
        return output

class FluidPINN(nn.Module):
    def __init__(self, layers):
        super(FluidPINN, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(len(layers) - 1):
            self.layers.append(nn.Linear(layers[i], layers[i+1]))

    def forward(self, x, y, t):
        inputs = torch.cat([x, y, t], dim=1)
        for i, layer in enumerate(self.layers[:-1]):
            inputs = torch.tanh(layer(inputs))
        output = self.layers[-1](inputs)
        u = output[:, 0:1]
        v = output[:, 1:2]
        p = output[:, 2:3]
        return u, v, p

class CoupledFSIPINN:
    def __init__(self, beam_layers, fluid_layers, beam_params, fluid_params):
        self.beam_net = BeamPINN(beam_layers)
        self.fluid_net = FluidPINN(fluid_layers)

        # Beam parameters
        self.rho_beam = beam_params['density']
        self.E = beam_params['youngs_modulus']
        self.I = beam_params['moment_inertia']
        self.A = beam_params['cross_section_area']
        self.L = beam_params['length']

        # Fluid parameters
        self.rho_fluid = fluid_params['density']
        self.nu = fluid_params['kinematic_viscosity']

        # Single optimizer for the entire system
        all_params = list(self.beam_net.parameters()) + list(self.fluid_net.parameters())
        self.optimizer = optim.Adam(all_params, lr=1e-3)

        # Store training history
        self.loss_history = {
            'beam_pde': [],
            'beam_bc': [],
            'fluid_pde': [],
            'fluid_bc': [],
            'total': []
        }

    def beam_pde_loss(self, x, t):
        x = x.clone().detach().requires_grad_(True)
        t = t.clone().detach().requires_grad_(True)

        u = self.beam_net(x, t)

        u_t = grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True)[0]
        u_x = grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]

        u_tt = grad(u_t, t, grad_outputs=torch.ones_like(u_t), create_graph=True)[0]
        u_xx = grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]

        u_xxx = grad(u_xx, x, grad_outputs=torch.ones_like(u_xx), create_graph=True)[0]
        u_xxxx = grad(u_xxx, x, grad_outputs=torch.ones_like(u_xxx), create_graph=True)[0]

        F_fluid = self.compute_fluid_force(x, t)
        pde_residual = self.rho_beam * self.A * u_tt + self.E * self.I * u_xxxx - F_fluid
        return torch.mean(pde_residual**2)

    def fluid_pde_loss(self, x, y, t):
        x = x.clone().detach().requires_grad_(True)
        y = y.clone().detach().requires_grad_(True)
        t = t.clone().detach().requires_grad_(True)

        u, v, p = self.fluid_net(x, y, t)

        u_t = grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True)[0]
        u_x = grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
        u_y = grad(u, y, grad_outputs=torch.ones_like(u), create_graph=True)[0]
        u_xx = grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]
        u_yy = grad(u_y, y, grad_outputs=torch.ones_like(u_y), create_graph=True)[0]

        v_t = grad(v, t, grad_outputs=torch.ones_like(v), create_graph=True)[0]
        v_x = grad(v, x, grad_outputs=torch.ones_like(v), create_graph=True)[0]
        v_y = grad(v, y, grad_outputs=torch.ones_like(v), create_graph=True)[0]
        v_xx = grad(v_x, x, grad_outputs=torch.ones_like(v_x), create_graph=True)[0]
        v_yy = grad(v_y, y, grad_outputs=torch.ones_like(v_y), create_graph=True)[0]

        p_x = grad(p, x, grad_outputs=torch.ones_like(p), create_graph=True)[0]
        p_y = grad(p, y, grad_outputs=torch.ones_like(p), create_graph=True)[0]

        momentum_x = u_t + u * u_x + v * u_y + p_x / self.rho_fluid - self.nu * (u_xx + u_yy)
        momentum_y = v_t + u * v_x + v * v_y + p_y / self.rho_fluid - self.nu * (v_xx + v_yy)
        continuity = u_x + v_y

        return torch.mean(momentum_x**2) + torch.mean(momentum_y**2) + torch.mean(continuity**2)

    def compute_fluid_force(self, x, t):
        return 0.1 * torch.sin(2 * np.pi * t) * torch.exp(-x/self.L)

    def beam_boundary_loss(self, t):
        t = t.clone().detach().requires_grad_(True)
        x_fixed = torch.zeros_like(t, requires_grad=True)

        u_fixed = self.beam_net(x_fixed, t)
        u_x_fixed = grad(u_fixed, x_fixed, grad_outputs=torch.ones_like(u_fixed), create_graph=True)[0]

        return torch.mean(u_fixed**2) + torch.mean(u_x_fixed**2)

    def fluid_boundary_loss(self, x_inlet, y_inlet, x_wall, y_wall, t_inlet, t_wall):
        u_inlet, v_inlet, _ = self.fluid_net(x_inlet, y_inlet, t_inlet)
        inlet_loss = torch.mean((u_inlet - 1.0)**2) + torch.mean(v_inlet**2)

        u_wall, v_wall, _ = self.fluid_net(x_wall, y_wall, t_wall)
        wall_loss = torch.mean(u_wall**2) + torch.mean(v_wall**2)

        return inlet_loss + wall_loss

    def compute_stress_field(self, x, t):
        x = x.clone().detach().requires_grad_(True)
        u = self.beam_net(x, t)

        u_x = grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
        u_xx = grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]

        y_max = 0.01
        stress = -self.E * y_max * u_xx
        return stress

    def train_step(self, beam_data, fluid_data):
        self.optimizer.zero_grad()

        x_beam, t_beam = beam_data['interior']
        t_boundary = beam_data['boundary']

        x_fluid, y_fluid, t_fluid = fluid_data['interior']
        x_inlet, y_inlet = fluid_data['inlet']
        x_wall, y_wall = fluid_data['wall']

        t_inlet = torch.rand_like(x_inlet) * 2.0
        t_wall = torch.rand_like(x_wall) * 2.0

        beam_pde_loss = self.beam_pde_loss(x_beam, t_beam)
        beam_bc_loss = self.beam_boundary_loss(t_boundary)
        fluid_pde_loss = self.fluid_pde_loss(x_fluid, y_fluid, t_fluid)
        fluid_bc_loss = self.fluid_boundary_loss(x_inlet, y_inlet, x_wall, y_wall, t_inlet, t_wall)

        total_loss = beam_pde_loss + 10.0 * beam_bc_loss + fluid_pde_loss + 10.0 * fluid_bc_loss

        total_loss.backward()
        self.optimizer.step()

        # Store loss history
        self.loss_history['beam_pde'].append(beam_pde_loss.item())
        self.loss_history['beam_bc'].append(beam_bc_loss.item())
        self.loss_history['fluid_pde'].append(fluid_pde_loss.item())
        self.loss_history['fluid_bc'].append(fluid_bc_loss.item())
        self.loss_history['total'].append(total_loss.item())

        return {
            'beam_pde_loss': beam_pde_loss.item(),
            'beam_bc_loss': beam_bc_loss.item(),
            'fluid_pde_loss': fluid_pde_loss.item(),
            'fluid_bc_loss': fluid_bc_loss.item(),
            'total_loss': total_loss.item()
        }

def generate_training_data(n_interior=1000, n_boundary=100):
    x_beam = torch.rand(n_interior, 1) * 1.0
    t_beam = torch.rand(n_interior, 1) * 2.0
    t_boundary = torch.rand(n_boundary, 1) * 2.0

    x_fluid = torch.rand(n_interior, 1) * 3.0 - 1.0
    y_fluid = torch.rand(n_interior, 1) * 2.0 - 1.0
    t_fluid = torch.rand(n_interior, 1) * 2.0

    x_inlet = torch.full((n_boundary, 1), -1.0)
    y_inlet = torch.rand(n_boundary, 1) * 2.0 - 1.0

    x_wall = torch.rand(n_boundary, 1) * 3.0 - 1.0
    y_wall = torch.full((n_boundary, 1), -1.0)

    beam_data = {
        'interior': (x_beam, t_beam),
        'boundary': t_boundary
    }

    fluid_data = {
        'interior': (x_fluid, y_fluid, t_fluid),
        'inlet': (x_inlet, y_inlet),
        'wall': (x_wall, y_wall)
    }

    return beam_data, fluid_data

def main():
    beam_layers = [2, 50, 50, 50, 1]
    fluid_layers = [3, 100, 100, 100, 3]

    beam_params = {
        'density': 7850.0,
        'youngs_modulus': 2e11,
        'moment_inertia': 8.33e-6,
        'cross_section_area': 1e-4,
        'length': 1.0
    }

    fluid_params = {
        'density': 1000.0,
        'kinematic_viscosity': 1e-6
    }

    print("Initializing Coupled FSI-PINN System...")
    fsi_pinn = CoupledFSIPINN(beam_layers, fluid_layers, beam_params, fluid_params)
    beam_data, fluid_data = generate_training_data()

    print("Starting Training...")
    epochs = 10000
    for epoch in range(epochs):
        try:
            losses = fsi_pinn.train_step(beam_data, fluid_data)

            if epoch % 1000 == 0:
                print(f"Epoch {epoch}:")
                print(f"  Beam PDE Loss: {losses['beam_pde_loss']:.6f}")
                print(f"  Beam BC Loss: {losses['beam_bc_loss']:.6f}")
                print(f"  Fluid PDE Loss: {losses['fluid_pde_loss']:.6f}")
                print(f"  Fluid BC Loss: {losses['fluid_bc_loss']:.6f}")
                print(f"  Total Loss: {losses['total_loss']:.6f}")
                print("-" * 50)

        except RuntimeError as e:
            print(f"Error at epoch {epoch}: {e}")
            break

if __name__ == "__main__":
    main()


Initializing Coupled FSI-PINN System...
Starting Training...
Epoch 0:
  Beam PDE Loss: 22192474112.000000
  Beam BC Loss: 0.020720
  Fluid PDE Loss: 0.004692
  Fluid BC Loss: 0.961056
  Total Loss: 22192474112.000000
--------------------------------------------------
Epoch 1000:
  Beam PDE Loss: 67272.242188
  Beam BC Loss: 0.001429
  Fluid PDE Loss: 0.011338
  Fluid BC Loss: 0.006163
  Total Loss: 67272.328125
--------------------------------------------------
Epoch 2000:
  Beam PDE Loss: 7426.083008
  Beam BC Loss: 0.001414
  Fluid PDE Loss: 0.007250
  Fluid BC Loss: 0.004125
  Total Loss: 7426.145508
--------------------------------------------------
Epoch 3000:
  Beam PDE Loss: 4165.585938
  Beam BC Loss: 0.001369
  Fluid PDE Loss: 0.007058
  Fluid BC Loss: 0.002939
  Total Loss: 4165.635742
--------------------------------------------------
Epoch 4000:
  Beam PDE Loss: 3271.940430
  Beam BC Loss: 0.001098
  Fluid PDE Loss: 0.005568
  Fluid BC Loss: 0.002698
  Total Loss: 3271.9838